# Project

The file `mRNA.fasta` contains a set of mRNA sequences expressed by a hypothetical organism. 
 
The objective of this project is to analyse these target sequences to discover what proteins the organism produces and how they differ from the corresponding proteins in other species.

#### 1. Coding sequence extraction

Extracts the coding sequences from the mRNA sequences.

a) For each target mRNA sequence, finds the longest open reading frame (ORF) in the sequence. An open reading frame is a continuous sequence that has the start codon `ATG` in the beginning and a stop codon at the end. Remember that the start codon can be at any position in the sequence.

b) Translates the ORFs to protein sequences for further analysis.

c) Saves the target protein sequences to the file `1-proteins.gb` in the GenBank format.

d) Summarises discoveries by reporting the position and length of the longest ORF for each target mRNA sequence.


In [1]:
## ORF OUT OF THE RNA SEQUENCES

# Import biopython and python modules
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import re
from Bio import SeqIO

from Bio import Seq

# Save as GenBank file
output_file = open('1-proteins.gb', 'w')

# File reader as fasta
for seq_record in SeqIO.parse("mRNA.fasta","fasta"):
    # Prints out the sequence IDS
    idSeq = seq_record.id
    print("Sequence name:",idSeq)
    print("Protein name:",idSeq)
    
    # Gets the sequence one at a time
    s = seq_record.seq
    
    # Have to be a string not a sequence object. Otherwise it would give an error
    madeintoString = str(s)                    
    
    # Start of the RNA ORF sequence
    startCodon = re.compile('AUG')
    
    # Sequence string
    nuc = madeintoString
    
    # Needed for comparison
    longest = (0,)
    
    # ORF finder
    # Used code by iayork --> https://stackoverflow.com/questions/31757876/python-find-longest-orf-in-dna-sequence
    # Modified to work for RNA and modified even more to ACTUALLY work, because without the modification the original script ...
    # ... does not work.
    
    # Uses regex to find the starting pattern in the sequence
    for m in re.finditer(startCodon, nuc):
        # Gives the longest translated protein out of every protein translated
        if len(Seq.Seq(nuc)[m.start():].translate(to_stop=True)) > longest[0]:
            # Translated protein
            pro = Seq.Seq(nuc)[m.start():].translate(to_stop=True)
            # This gives the lenght of the translated protein, that starting index of the ORF (AUG codon), 
            # the ending index of the ORF (without the position of the stopcodon included) and the protein itself
            longest = (len(pro), 
                       m.start(), 
                       len((nuc[m.start():m.start()+len(pro)*3]))+m.start(),
                       str(pro))
     
    # Protein lenght
    lenght = longest[0]
    print("Protein lenght:",lenght)
    
     # Start of ORF
    startORF = longest[1]
    print("The starting position of ORF in RNA sequence:",startORF)
    
    # End of ORF
    endORF = longest[2]
    print("The ending position of ORF in RNA sequence:",endORF)
    
    # Protein sequence
    prot = longest[3]
    print("Translated protein:",prot)
    print()
   
    # Below is writing the file in genbank format
    # Create a sequence
    sequence_object = Seq.Seq(prot, IUPAC.extended_protein)
 
    # Create a record
    record = SeqRecord(sequence_object,
                            id = idSeq, # random accession number which is mRNA in our case
                            name = idSeq)
    # Write to file
    SeqIO.write(record, output_file, 'gb')

# Close the file
output_file.close()



Sequence name: mRNA_1
Protein name: mRNA_1
Protein lenght: 616
The starting position of ORF in RNA sequence: 437
The ending position of ORF in RNA sequence: 2285
Translated protein: MSGKAQQQSRIKELIVLGREQGYLTYAEVNDHLPEDISDPEQVEDIIRMINDMGINVFEAAPDKDSLMLADADTDEAAAEEAAAALAAVETDIGRTTDPVRMYMREMGTVELLTREGEIEIAKRIEEGIREVMGAIAHFPGTVDHILSEYTRVTTEGGRLSDVLSGYIDPDDGIAPPAAEVPPPVGAKTAKADDDTDDDEAEASSDDEEEVESGPDPIIAAQRFGAVSDQMEITRKALKKHGRGNKQAIAELIALAELFMPIKLVPKQFEGLVERVRSALERLRAQERAIMQLCVRDARMPRADFVRQFPGNEVDESWTDALAKGKAKYAEAIGRLQPDIIRCQQKLTALETETGLTIAEIKDINRRMSIGEAKARRAKKEMVEANLRLVISIAKKYTNRGLQFLDLIQEGNIGLMKAVDKFEYRRGYKFSTYATWWIRQAITRSIADQARTIRIPVHMIETINKLNRISRQMLQEMGREPTPEELGERMEMPEDKIRKVLKIAKEPISMETPIGDDEDSHLGDFIEDSTMQSPIDVATVESLKEATRDVLSGLTAREAKVLRMRFGIDMNTDHTLEEVGKQFDVTRERIRQIEAKALRKLRHPTRSEHLRSFLDE

Sequence name: mRNA_2
Protein name: mRNA_2
Protein lenght: 372
The starting position of ORF in RNA sequence: 42
The ending position of ORF in RNA sequence: 1158
Translated protein: MTVTPERSTYTPPGRLGDE

D:\Programs\Anaconda3\lib\site-packages\Bio\Seq.py:2715: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


<b><u>Report/Disscussion</u></b><p>

After the code has completed its run, we can see that the translated ORF of every mRNA have very different protein lenghts. mRNA_4 being the longest protein and mRNA_3 being the shortest of them all.<br> 
Because the original sequence is a RNA sequence, there is no need to find the reverse complement for the ORF because RNA is not double helical.<br>

The actual approach was that first the code must read every RNA sequence from a FASTA file. The FASTA file has serveral mRNA sequences. There we can get the individual mRNA sequence and its ID. To translate the protein, its open reading frame (ORF) is needed, which in RNA's case starts with AUG codon. The actual ORF starts with the AUG and ends with its end codon, but in actual translation the end codon is not included. So we must skim trough every individual sequence and find its longest ORF. The approach is a loop. I goes through the individual mRNA sequence, looks for the AUG codon and a stop codon (from translate table, which is provided by BioPython, but since we are using the 1st one which is a starndar codon table we don't have to specify it, BioPython assumes we use the first 1 by default). After finding every ORF possible we look for the longest one and then translate it to get a protein.<br>
Then all of the translated proteins are written into a genbank file by using BioPython provided methods. 

#### 2. Similarity search

Finds similar proteins from the NCBI Protein database.

a) For each target protein sequence in the task 1, runs BLAST to obtain similar sequences. Saves the BLAST results to the file `2-blast.xml`.

b) Selects the hits containing a high-scoring pair (HSP) with the following properties:
- has E-value $\leq 10^{-6}$
- spans $\geq 75\%$ of the target sequence
- percent identity $\geq 30\%$

c) Updates the records of the target protein sequences by adding the database ids of the selected hits. Uses a sequence feature with the type `BLAST` and the qualifier key `db_xref` containing a list of ids. Saves the updated records to the file `2-proteins.gb`.

d) Summarises discoveries by reporting the ids of the selected hits for each target protein sequence.

In [2]:
## GENBANK FILE CONVERTION INTO FASTA FILE

# Import biopython and python modules
from Bio import SeqIO

# Turns genbank file into Fasta
# The code snipit is taken from biopython.org:
# link --> https://biopython.org/wiki/Converting_sequence_files
with open("1-proteins.gb", "rU") as input_handle:
    with open("1-proteins.fasta", "w") as output_handle:
        sequences = SeqIO.parse(input_handle, "genbank")
        count = SeqIO.write(sequences, output_handle, "fasta")

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 'U' mode is deprecated
  if __name__ == '__main__':


In [3]:
## BLAST RUN 

# Import biopython and python modules
from Bio.Blast import NCBIWWW

## Opens the fasta file and uses it to protein BLAST the sequences in it
# For every proten sequence
fastaFile = open("1-proteins.fasta","r")
# Main line of code where the pBLAST happens
result_handle = NCBIWWW.qblast("blastp", "swissprot", fastaFile.read())
fastaFile.close()

# Writes the protein BLAST results into an xml file
with open("2-blast.xml", "w") as file:
    file.write(result_handle.read())


In [4]:
## BLAST HIT READING

# Import bipython modules
import Bio.SearchIO as BSIO

# Reads all of the pBLAST results that are in the xml file
with open('2-blast.xml') as f:
    for result in BSIO.parse(f, 'blast-xml'):
        print(result)

Program: blastp (2.10.0+)
  Query: mRNA_1 (616)
 Target: swissprot
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  sp|P52326.1|  RecName: Full=RNA polymerase sigma factor...
            1      1  sp|P26480.1|  RecName: Full=RNA polymerase sigma factor...
            2      1  sp|P52327.1|  RecName: Full=RNA polymerase sigma factor...
            3      1  sp|P0A2E3.1|  RecName: Full=RNA polymerase sigma factor...
            4      1  sp|P00579.2|  RecName: Full=RNA polymerase sigma factor...
            5      1  sp|O24744.1|  RecName: Full=RNA polymerase sigma factor...
            6      1  sp|P32001.1|  RecName: Full=RNA polymerase sigma factor...
            7      1  sp|P57163.1|  RecName: Full=RNA polymerase sigma factor...
            8      1  sp|Q89B10.1|  RecName: Full=RNA polymerase sigma factor...
   

In [6]:
## FILTERING BLAST HITS AND FILE WRITING

# Import biopython and python modules
from Bio import SeqIO
import Bio.SearchIO as BSIO
from Bio import SeqFeature
from Bio.SeqFeature import SeqFeature
from Bio import SeqFeature as sf
import Bio.SeqFeature as BSF

# Visas tas sarasas tu proteinu
# All of the existing genbank mRNAs with their protein
proteins = list(SeqIO.parse("1-proteins.gb","gb"))
#print(proteins)

# Function for e-value that is lest or equel to 10e-6
def evalue(hsp):
    return hsp.evalue <= 0.000001
     
# Function for span that is more or equel to 75 percent of the target sequence
def span(hsp):
    return (hsp.query_span / seq_len) * 100 >= 75

# Function for percent identity that is more or equal to 30
def identity(hsp):
    return (hsp.ident_num / hsp.aln_span) * 100 >= 30

# Main code that reads the file 
with open('2-blast.xml') as file:
    for result in BSIO.parse(file, 'blast-xml'):
        # Filteres all of the results that have that evalue()
        filtered_result = result.hsp_filter(evalue)
        # Sequnce lenghts with that evalue
        seq_len = filtered_result.seq_len
        # Filteres out the hits that have that evalue() filtered before but this time looks at the span()
        results_forvalueNspan = filtered_result.hsp_filter(span)
        # Filteres out the hits that have that evalue() and span() like filtered before but this time looks at the identity()
        results_forAll = results_forvalueNspan.hsp_filter(identity)
        #for hit in results_forAll:
            #print(hit)
        
        # every mRNA result ID name and so on from BLAST
        rnaName = result.id
        print("Protein name:",rnaName)
        
        # Prints out all of the results that were filtered
        db_xrefs_seq_ids = []
        
        # Gets all of the hit IDs without the sp|| around it. Thats why we use accession
        # Appends into a list
        for hit in results_forAll:
            idd = hit.accession
            db_xrefs_seq_ids.append(idd)
        print("Blast hit ID's:",db_xrefs_seq_ids)
        print()
            
        # Check if we found it or not
        protein = None
        
        # jei liste tas mano s turi ta name = ""
        # So for every protein or that mRNA1 in all of the proteins or alll of the mRNAs in the file we look 
        # if the name  of the protein in genbank file is the same as the name in the BLAST results of hits
        # proteins is our genbank file
        for p in proteins:
            if p.name == rnaName:
                protein = p
                break
        #print(protein)
            
        # Creation of BLAST feature for our protein sequences'
        # Len vieno protein o ne visu proteins yra tas 1..616 salia BLAST
        feature = BSF.SeqFeature(BSF.FeatureLocation(0,len(protein)), type='BLAST')
        # We add a qualifier 'db_xref' to the hit ids (accenssions) and add it to BLAST feature
        feature.qualifiers['db_xref'] = db_xrefs_seq_ids
        
        # Then for every protein (not the whole stack of proteins but just every individual protein/mRNA)in the genbank file 
        # we add those features that have a name BLAST on the left collum
        # and we add all of the feature items, I mean all of those ids with the qualifier
        protein.features.append(feature)
        #print(features)

# Write to genbank file
SeqIO.write(proteins, "2-proteins.gb","gb")
      

Protein name: mRNA_1
Blast hit ID's: ['P52326', 'P26480', 'P52327', 'P0A2E3', 'P00579', 'O24744', 'P32001', 'P57163', 'Q89B10', 'Q83BB6', 'Q9PDM9', 'Q87DT7', 'Q8PG33', 'P43766', 'Q8P4H2', 'P52325', 'P33452', 'Q59753', 'Q2K619', 'P52324', 'D5AQI9', 'P17531', 'P0CZ15', 'Q1RKH7', 'Q92FZ8', 'Q4UJT1', 'P33451', 'Q68VQ5']

Protein name: mRNA_2
Blast hit ID's: []

Protein name: mRNA_3
Blast hit ID's: ['Q9RR29', 'D4GU70', 'P08075', 'A6VG23', 'A6UP85', 'A4FX98', 'Q58501', 'Q6LYB5', 'A6UUQ4', 'Q975F9', 'Q9Y725', 'Q6FRY2', 'P0C5I2', 'O93827', 'Q8BTZ7', 'Q9Y5P6', 'Q752H4', 'P41940', 'Q6BN12', 'O74624', 'Q2YDJ9', 'A2VD83', 'Q4U3E8']

Protein name: mRNA_4
Blast hit ID's: ['P14738', 'Q2FE03', 'Q6G6H3', 'A7X6I5', 'Q6GDU5', 'Q2YW62', 'A0A0H2XKG3']

Protein name: mRNA_5
Blast hit ID's: ['P08320', 'P39347', 'P37317', 'P37326', 'P32053', 'P76542']

Protein name: mRNA_6
Blast hit ID's: ['P69451', 'Q8XDR6', 'P63521', 'Q8ZES9', 'P46450', 'P94547', 'O07610', 'Q0DV32', 'P41636', 'Q42982', 'Q54P79', 'Q54P77', '

6

<b><u>Report/Disscussion</u></b><p>
As we can see after the run that only the mRNA2 protein does not have BLAST IDs based on our criteria.<p>
The base idea of the code was to find BLAST hits that are based on 3 properties.<br>
First I converted the GenBank file into FASTA file to get the sequences we need. Then for every mRNA protein sequence I ran protein BLAST in swissprot library. Then I checked the given blastp results. To get the hits based on our criteria I made functions for every property like: evalue for e-value, span for aligned sequence span and identity for sequence similarity. The functions make the code more understandable. I used those functions to filter out the results that are stored in the blast xml file.<br>
Then I took out just the IDs from our filtered BLAST results. Then I checked if our filtered BLAST results have the same query name like the ones in our genbank file. Basically for example if mRNA_1 in BLAST results equel to mRNA1 in our genbank file.<br>
Later I created a new genbank file sequence Feature called BLAST, there I add qualifiers that are our BLAST IDs. Then append those features to every protein in genbank file. And then parse it into a new file: take the information form our 1st genbank file and make a new one (2nd file).

#### 3. Protein function prediction

Infering of the putative functions by similarity.

a) Downloads the UniProt entries of the selected hits in the task 2. Saves the entries to the files `XXXXXX.xml` where `XXXXXX` is the UniProt id.

b) Separately for each target protein sequence in the task 1, finds those Gene Ontology (GO) terms which occur `in all` of the matched UniProt entries.

c) Updates the records of the target protein sequences by adding the found GO terms. Uses a sequence feature with the type GO and the qualifier key `db_xref` containing a list of ids. Also adds the qualifier `evidence` with value `inferred by similarity` to the feature. Saves the updated records to the file `3-proteins.gb`.

d) Summarises discoveries by reporting the putative GO terms for each target protein sequence. Prints both the GO term ids and their human-friendly names.

In [7]:
## ## READS GENBANK FILE AND SAVES BLAST IDS INTO AN XML FILE

# Import biopython and python modules
from Bio import SeqIO
import collections as C
import requests as R
import itertools
import collections
import Bio.SeqFeature as BSF

proteins2 = list(SeqIO.parse("2-proteins.gb","gb"))
#print(proteins2)

for protein in proteins2:
    #print(protein)
    # Finds all of the ID that are in genbank file 2-proteins-tryout with feature - BLAST
    for f in protein.features:
        if f.type == "BLAST":
            if "db_xref" in f.qualifiers:
                idss = f.qualifiers["db_xref"]
                print(idss)
                
                
                # By the individual ID get the UniProt URLs and download the entries as XML files
                for idd in idss:
                    # Prints ids just to check how they look
                    print(idd)

                    # API address
                    base = "https://www.uniprot.org/uniprot/%s.%s"

                    # Format
                    fmt = 'xml'

                    # Adds the base url to idd and xml format by replaces the %s.%s
                    # It would be something like this if the id (idd) is 11:
                    # https://www.uniprot.org/uniprot/11.xml
                    url = base%(idd, fmt)

                    # Gets the URL response so that the contents of the URL could be saved
                    response = R.get(url)

                    # Prints URLs just to see how it came out
                    print(url)
                    
                    # For every id (idd) uniprot id we have, save the id's content into an XML file.
                    # So let's say our id (idd) is 11
                    # Then the xml file name would be 11.xml and the https://www.uniprot.org/uniprot/11.xml content ...
                    # ... would be save into that 11.xml file
                    #
                    # Opens file
                    output_file = open(f'{idd}.xml', 'w')
                    # Writes to file
                    output_file.write(response.text)
                    # Closes the file
                    output_file.close()

['P52326', 'P26480', 'P52327', 'P0A2E3', 'P00579', 'O24744', 'P32001', 'P57163', 'Q89B10', 'Q83BB6', 'Q9PDM9', 'Q87DT7', 'Q8PG33', 'P43766', 'Q8P4H2', 'P52325', 'P33452', 'Q59753', 'Q2K619', 'P52324', 'D5AQI9', 'P17531', 'P0CZ15', 'Q1RKH7', 'Q92FZ8', 'Q4UJT1', 'P33451', 'Q68VQ5']
P52326
https://www.uniprot.org/uniprot/P52326.xml
P26480
https://www.uniprot.org/uniprot/P26480.xml
P52327
https://www.uniprot.org/uniprot/P52327.xml
P0A2E3
https://www.uniprot.org/uniprot/P0A2E3.xml
P00579
https://www.uniprot.org/uniprot/P00579.xml
O24744
https://www.uniprot.org/uniprot/O24744.xml
P32001
https://www.uniprot.org/uniprot/P32001.xml
P57163
https://www.uniprot.org/uniprot/P57163.xml
Q89B10
https://www.uniprot.org/uniprot/Q89B10.xml
Q83BB6
https://www.uniprot.org/uniprot/Q83BB6.xml
Q9PDM9
https://www.uniprot.org/uniprot/Q9PDM9.xml
Q87DT7
https://www.uniprot.org/uniprot/Q87DT7.xml
Q8PG33
https://www.uniprot.org/uniprot/Q8PG33.xml
P43766
https://www.uniprot.org/uniprot/P43766.xml
Q8P4H2
https://www.

In [4]:
## READS BLAST XML FILE AND SAVES BLAST IDS INTO AN XML FILE

# Import biopython and python modules
import requests as R
from Bio import SeqIO
import Bio.SearchIO as BSIO
from Bio import SeqFeature
from Bio.SeqFeature import SeqFeature
from Bio import SeqFeature as sf
import Bio.SeqFeature as BSF 

# Function for e-value that is less or equel to 10e-6
def evalue(hsp):
    return hsp.evalue <= 0.000001
     
# Function for span that is more or equel to 75 percent of the target sequence
def span(hsp):
    return (hsp.query_span / seq_len) * 100 >= 75

# Function for percent identity that is more or equal to 30
def identity(hsp):
    return (hsp.ident_num / hsp.aln_span) * 100 >= 30

# Main code that reads the file 
with open('2-blast.xml') as file:
    for result in BSIO.parse(file, 'blast-xml'):
        # Filteres all of the results that have that evalue()
        filtered_result = result.hsp_filter(evalue)
        # Sequnce lenghts with that evalue
        seq_len = filtered_result.seq_len
        # Filteres out the hits that have that evalue() filtered before but this time looks at the span()
        results_forvalueNspan = filtered_result.hsp_filter(span)
        # Filteres out the hits that have that evalue() and span() like filtered before but this time looks at the identity()
        results_forAll = results_forvalueNspan.hsp_filter(identity)
        #for hit in results_forAll:
            #print(hit)
        
        # every mRNA1 name and so on from BLAST
        rnaName = result.id
        print("RNA name:",rnaName)
        
        # Prints out all of the results that were filtered
        db_xrefs_seq_ids = []
        
        # Gets all of the hit IDs withou th sp|| around it. Thats why we use accession
        # Appends into a list
        for hit in results_forAll:
            idd = hit.accession
            db_xrefs_seq_ids.append(idd)
        print("Blast hit ID's:",db_xrefs_seq_ids)
        
        # By the individual ID get the UniProt URLs and download the entries as XML files
        for idd in db_xrefs_seq_ids:
            # Prints ids just to check how they look
            print(idd)
        
            # API address
            base = "https://www.uniprot.org/uniprot/%s.%s"
            
            # Format
            fmt = 'xml'
            
            # Adds the base url to idd and xml format by replaces the %s.%s
            # It would be something like this if the id (idd) is 11:
            # https://www.uniprot.org/uniprot/11.xml
            url = base%(idd, fmt)
            
            # Gets the URL response so that the contents of the URL could be saved
            response = R.get(url)
            
            # Prints URLs just to see how it came out
            print(url)
        
            # For every id (idd) uniprot id we have, save the id's content into an XML file.
            # So let's say our id (idd) is 11
            # Then the xml file name would be 11.xml and the https://www.uniprot.org/uniprot/11.xml content ...
            # ... would be save into that 11.xml file
            #
            # Opens file
            output_file = open(f'{idd}.xml', 'w')
            # Writes to file
            output_file.write(response.text)
            # Closes the file
            output_file.close()

RNA name: mRNA_1
Blast hit ID's: ['P52326', 'P26480', 'P52327', 'P0A2E3', 'P00579', 'O24744', 'P32001', 'P57163', 'Q89B10', 'Q83BB6', 'Q9PDM9', 'Q87DT7', 'Q8PG33', 'P43766', 'Q8P4H2', 'P52325', 'P33452', 'Q59753', 'Q2K619', 'P52324', 'D5AQI9', 'P17531', 'P0CZ15', 'Q1RKH7', 'Q92FZ8', 'Q4UJT1', 'P33451', 'Q68VQ5']
P52326
https://www.uniprot.org/uniprot/P52326.xml
P26480
https://www.uniprot.org/uniprot/P26480.xml
P52327
https://www.uniprot.org/uniprot/P52327.xml
P0A2E3
https://www.uniprot.org/uniprot/P0A2E3.xml
P00579
https://www.uniprot.org/uniprot/P00579.xml
O24744
https://www.uniprot.org/uniprot/O24744.xml
P32001
https://www.uniprot.org/uniprot/P32001.xml
P57163
https://www.uniprot.org/uniprot/P57163.xml
Q89B10
https://www.uniprot.org/uniprot/Q89B10.xml
Q83BB6
https://www.uniprot.org/uniprot/Q83BB6.xml
Q9PDM9
https://www.uniprot.org/uniprot/Q9PDM9.xml
Q87DT7
https://www.uniprot.org/uniprot/Q87DT7.xml
Q8PG33
https://www.uniprot.org/uniprot/Q8PG33.xml
P43766
https://www.uniprot.org/unipr

In [8]:
import re as R
import collections as C

# matches stanza header
r_header = R.compile(r'^\[(\w+)\]$')
# matches tag:value
r_tag = R.compile(r'^(.*?): (.*)( ! .*)?$')
# named tuple
Term = C.namedtuple('Term', ['id', 'name'])

def parse_go_obo(go_obo_file):
    # result dictionary
    go = {}
    # current stanza data
    head = ''
    data = {}
    # process one line at the time
    with open(go_obo_file, 'rt') as f:
        for line in f:
            # strip newline
            line = line.strip()
            # skip over empty lines
            if not line:
                continue
            # check if new stanza is encountered
            header = r_header.match(line)
            if header:
                # flush current term if any
                if head == 'Term' and len(data) == 2:
                    term = Term(**data)
                    go[term.id] = term
                # start new stanza
                head = header.group(1)
                data = {}
            else:
                # collect data
                tag, value, _ = r_tag.match(line).groups()
                # store only needed tags
                if tag in ['id', 'name']:
                    data[tag] = value
        # flush the last term
        if head == 'Term' and len(data) == 2:
            term = Term(**data)
            go[term.id] = term
    # return result
    return go

go = parse_go_obo('go.obo')

# (just a brief check what is in dictionary)
k = go['GO:0000001']
for i in k:
    print(i)
#print(go['GO:0000001'])

GO:0000001
mitochondrion inheritance


In [9]:
## GO TERM RETRIEVAL BASED ON OCCURANCE IN SEVERAL BLAST IDS

from Bio import SeqIO
import collections as C
import requests as R
import itertools
import collections
from collections import Counter
import Bio.SeqFeature as BSF

# Our genbank file
proteins2 = list(SeqIO.parse("2-proteins.gb","gb"))
#print(proteins2)

# Skimming trough all of the protein information in the 2-proteins.gb file
# Goes 1 by 1
for protein in proteins2:
    
    print()
    
    # Name of protein
    names = "Protein:" + protein.name
    print(names)
    
    # Underline of protein name
    print(len(names)*'-')
    #print(protein)
    
    # Finds all of the IDs that are in genbank file 2-proteins with feature - BLAST
    for f in protein.features:
        # Check if the feature in genbank file is BLAST
        if f.type == "BLAST":
            # Check if qualifier is db_xref, which has our BLAST IDs.
            if "db_xref" in f.qualifiers:
                # Get all of those IDs
                idss = f.qualifiers["db_xref"]
                #print(idss)
                
                # Lenght of how many BLAST IDs we have for every protein mRNA_
                lenIDs = len(idss)
                #print(lenID)
                
                # Empty list to store ALL GO terms from ALL of the xml files
                # It's gonna be a list of lists like so: [[],[],...]
                goTerms = []
                
                # For every ID in all genbank file BLAST IDs ...
                for idd in idss:
                    
                    # Read the xml files which have the genbank BLAST IDs that we took to download our uniprot xml files before
                    # So for every BLAST ID in the in genbank file we read an .xml file associated with that ID
                    fullProtein = SeqIO.read(f'{idd}.xml','uniprot-xml')
                    #print(fullProtein)
    
                    # Gets all of the GO IDs from ALL of the XML files we read before
                    for ref in fullProtein.dbxrefs:
                        # Cuts the ID because if not, it would start as GO:GO----
                        goTerm = ref[3:]
                        # If the ref (dbReference in xml file) starts with GO, append it to the list of lists
                        if ref.startswith('GO:'):
                            #print(goTerm)
                            goTerms.append(goTerm)
                #print(goTerms)
                
                # Count how many of every single term there is
                # For example GO: 1 occurs 24 times, GO: 2 occurs 10 times and so on
                dictt = Counter(goTerms)
                #print(dictt)
                
                # Empty list for all of the Terms that we need
                # All of them that occurs in all of the BLAST IDs we have for every protein mRNA_
                AllTerms = []
                
                # Here we loop trough out dictt with the GO terms counted
                # We get the second item of the dictionary: 'GO: 1': 7, so we get the 7
                # And we look if the second item, for eg.: that 7, is equal to all of the BLAST IDs for a protein, the lenght of BLAST IDs
                # item1 is our GO and item2 is our number (how many times it occurs)
                # We append it to AllTerms list only those GO terms that occur in all of the BLAST IDs
                # If we have 7 blast ids and we get GO that occurs 7, we append it
                # If we got a GO that occured 6 times we dont, because it occured only in 6 BLAST IDs and not all 7
                for item1, item2 in dictt.items():
                    if item2 == lenIDs:
                        AllTerms.append(item1)
                #print(AllTerms)
                
                # Modified duplicate finder by georg --> https://stackoverflow.com/questions/9835762/how-do-i-find-the-duplicates-in-a-list-and-create-another-list-with-them
                # Terms Gets only the duplicates in every list
                # Not what we need so its incorrect
                #Terms = [item for item, count in collections.Counter(goTerms).items() if count > 1]
                #print(Terms)
                
                # For every term in Terms variable we get the human readable names of it using parse_go_obo() function
                # And then print it out nicely
                for everyGo in AllTerms:
                    # go[] is the parser
                    content = go[everyGo]
                    for stuff in content:
                        print(stuff)
                    print()
                
                # Creation of GO feature for our protein sequences'
                # Len vieno protein o ne visu proteins yra tas 1..616 salia BLAST
                feature = BSF.SeqFeature(BSF.FeatureLocation(0,len(protein)), type='GO')
                
                # We add a qualifier 'db_xref' to the GO IDs (Terms) and add it to GO feature
                feature.qualifiers['db_xref'] = AllTerms
                
                # In the same GO feature we add location/qualifier that is 'evidence'
                feature.qualifiers['evidence'] = 'inferred by similarity'
        
                # Then for every protein (not the whole stack of proteins but just every individual protein/mRNA)in the genbank file 
                # we add those features that have a name GO on the left collum
                # and we add all of the feature items, I mean all of those ids with the qualifier
                protein.features.append(feature)
                #print(features)
                
# Write to genbank file
SeqIO.write(proteins2, "3-proteins.gb","gb")


Protein:mRNA_1
--------------
GO:0001123
transcription initiation from bacterial-type RNA polymerase promoter

GO:0003677
DNA binding

GO:0003700
DNA-binding transcription factor activity

GO:0016987
sigma factor activity


Protein:mRNA_2
--------------

Protein:mRNA_3
--------------

Protein:mRNA_4
--------------
GO:0005576
extracellular region

GO:0005618
cell wall

GO:0007155
cell adhesion

GO:0009405
pathogenesis

GO:0016020
membrane


Protein:mRNA_5
--------------
GO:0044826
viral genome integration into host DNA

GO:0046718
viral entry into host cell


Protein:mRNA_6
--------------
GO:0005524
ATP binding



6

<b><u>Report/Disscussion</u></b><p>
The whole idea was to get GO terms for our protein sequences and get the human readable names.<p>
So firstly we need to download the UniProt files for every ID that I have gotten after running BLAST. I made two ways you can do that: from our already made genbank file and from our BLAST xml. The code downloads all of the UniProt files in .xml format for every BLAST ID. Then the code reads all of the .xml files coresponding to the protein (mRNA) it was assigned to in BLAST and gets the GO terms. Since the `for loop` loops and gets all of the .xml file names or the BLAST IDs to its corresponding mRNA, that means that all of the GO terms found are stored into a list. For exmaple: for mRNA_1 we have a list with GO terms from certain .xmls, then for mRNA_2 and so on. So my idea was to put ALL of the GO terms found in EVERY .xml file and just keep the the GO terms that occur in every .xml file. So every list [] of GO terms is put into a list of lists [[],[],...]. The it that list get `Counter()` function called upon and that function counts how many of times it occured is a value. Afte that, we check if the counted number (occurancen number or value) is equal to our number of BLAST IDs for a singal query/original protein. So if we for mRNA_1 we have 6 BLAST IDs then we get how many we have, and compare to the value of our new dictionary. If so, then that means, that those GO terms appear in every BLAST .xml file for that protein. Then we put it into a list. <br>
To get the human readable names I need a `go.obo` file which is provided by the UniProt already. I used a parses that was made in the exercize 4, which parses the `go.obo`. So I just had to use the parser and parse it by the GO terms I have gotten.<p>
We can seee that mRNA_2 does not have any GO terms since there were no BLAST IDs based on our criteria, and also mRNA_3 has no GO terms. mRNA_4 has the most GO terms, while mRNA_6 has the least. 

#### 4. Sequence alignment

Finds the positions at which the sequences differ from consensus.

a) For each target protein sequence in the task 1, aligns the target sequence with the sequences of the UniProt entries in the task 3. Saves the alignments to the file `3-alignment-X.fasta` in the FASTA format where `X` is a running number.

b) Finds the positions in the alignments at which the target sequence is not identical to the consensus. Uses 60% consensus (i.e. the consensus exists if at least 60% of the aligned sequences contain the same residue). Ignores the non-consensus positions and the gaps in either sequence.

c) Summarises discoveries by reporting the positions where the target protein sequence is not identical to the consensus sequence for each alignment.

d) Updates the records of the target protein sequences by adding the non-identical positions. Uses a sequence feature with the type `mutation` and the qualifier keys `observed` and `consensus` to store the residues in the target sequence and the consensus sequence, respectively. Saves the updated records to the file `4-proteins.gb`.


In [10]:
## DOWNLOADS ALL OF THE SEQENCES OF BLAST IDS AND SAVES IT INTO THEIR SPECIFIC FILE

# Import biopython and python modules
from Bio import SeqIO
import collections as C
import requests as R
import itertools
import collections
import Bio.SeqFeature as BSF
from pathlib import Path

# It prints out a list of IDS and then one by one the link of the ID.
# It helps to indicate which one is being written and how much are left.

proteins2 = list(SeqIO.parse("2-proteins.gb","gb"))
#print(proteins2)

for protein in proteins2:
     
    # Protein name like mRNA and so on
    pname = protein.id
    
    # Original protein sequence
    pseq = protein.seq
    
    # Opens file
    output_file = open(f'{pname}.fasta', 'w')
    
    #print(protein)
    # Finds all of the ID that are in genbank file 2-proteins-tryout with feature - BLAST
    for f in protein.features:
        if f.type == "BLAST":
            if "db_xref" in f.qualifiers:
                idss = f.qualifiers["db_xref"]
                print(idss)
                
                # By the individual ID get the UniProt URLs and download the entries as XML files
                for idd in idss:
                    # Prints ids just to check how they look
                    print(idd)

                    # API address
                    base = "https://www.uniprot.org/uniprot/%s.%s"

                    # Format
                    fmt = 'fasta'

                    # Adds the base url to idd and xml format by replaces the %s.%s
                    # It would be something like this if the id (idd) is 11:
                    # https://www.uniprot.org/uniprot/11.xml
                    url = base%(idd, fmt)

                    # Gets the URL response so that the contents of the URL could be saved
                    response = R.get(url)

                    # Prints URLs just to see how it came out
                    print(url)

                    # For every id (idd) uniprot id we have, save the id's content into an XML file.
                    # So let's say our id (idd) is 11
                    # Then the xml file name would be 11.xml and the https://www.uniprot.org/uniprot/11.xml content ...
                    # ... would be save into that 11.xml file
                    #
                    # Writes to file
                    output_file.write(response.text)
                    
    # Closes the file
    output_file.close() 

['P52326', 'P26480', 'P52327', 'P0A2E3', 'P00579', 'O24744', 'P32001', 'P57163', 'Q89B10', 'Q83BB6', 'Q9PDM9', 'Q87DT7', 'Q8PG33', 'P43766', 'Q8P4H2', 'P52325', 'P33452', 'Q59753', 'Q2K619', 'P52324', 'D5AQI9', 'P17531', 'P0CZ15', 'Q1RKH7', 'Q92FZ8', 'Q4UJT1', 'P33451', 'Q68VQ5']
P52326
https://www.uniprot.org/uniprot/P52326.fasta
P26480
https://www.uniprot.org/uniprot/P26480.fasta
P52327
https://www.uniprot.org/uniprot/P52327.fasta
P0A2E3
https://www.uniprot.org/uniprot/P0A2E3.fasta
P00579
https://www.uniprot.org/uniprot/P00579.fasta
O24744
https://www.uniprot.org/uniprot/O24744.fasta
P32001
https://www.uniprot.org/uniprot/P32001.fasta
P57163
https://www.uniprot.org/uniprot/P57163.fasta
Q89B10
https://www.uniprot.org/uniprot/Q89B10.fasta
Q83BB6
https://www.uniprot.org/uniprot/Q83BB6.fasta
Q9PDM9
https://www.uniprot.org/uniprot/Q9PDM9.fasta
Q87DT7
https://www.uniprot.org/uniprot/Q87DT7.fasta
Q8PG33
https://www.uniprot.org/uniprot/Q8PG33.fasta
P43766
https://www.uniprot.org/uniprot/P437

In [8]:
## COMBINES THE ORIGINAL PROTEIN SEQUENCE TO ITS DEDICATED BLAST SEQUENCES

from Bio import SeqIO

# For every our original mRNA, write it into a file with the same name.
# Write the original protein sequence and then the BLAST hit sequences
# For eg.: mRna_1 and then all BLAST ID corresponding to it, and then mRNA_2 and so on. 
for record in SeqIO.parse("1-proteins.fasta", "fasta"):
    
    # Name of the protein
    name = record.id
    
    # Read the .fasta files we downloaded in the previous cell
    recordFasta = list(SeqIO.parse(f'{name}.fasta','fasta'))
    #print(recordFasta)
    
    records = [record] + recordFasta
    #print(records)
    
    # Write into a new file our original protein sequence and the BLAST hit sequences
    SeqIO.write(records,f'{name}_new.fasta','fasta')

In [9]:
# MUSCLE help

from Bio.Align.Applications import MuscleCommandline
from Bio import AlignIO

help(MuscleCommandline)

Help on class MuscleCommandline in module Bio.Align.Applications._Muscle:

class MuscleCommandline(Bio.Application.AbstractCommandline)
 |  MuscleCommandline(cmd='muscle', **kwargs)
 |  
 |  Command line wrapper for the multiple alignment program MUSCLE.
 |  
 |  http://www.drive5.com/muscle/
 |  
 |  Notes
 |  -----
 |  Last checked against version: 3.7, briefly against 3.8
 |  
 |  References
 |  ----------
 |  Edgar, Robert C. (2004), MUSCLE: multiple sequence alignment with high
 |  accuracy and high throughput, Nucleic Acids Research 32(5), 1792-97.
 |  
 |  Edgar, R.C. (2004) MUSCLE: a multiple sequence alignment method with
 |  reduced time and space complexity. BMC Bioinformatics 5(1): 113.
 |  
 |  Examples
 |  --------
 |  >>> from Bio.Align.Applications import MuscleCommandline
 |  >>> muscle_exe = r"C:\Program Files\Aligments\muscle3.8.31_i86win32.exe"
 |  >>> in_file = r"C:\My Documents\unaligned.fasta"
 |  >>> out_file = r"C:\My Documents\aligned.fasta"
 |  >>> muscle_cli

In [10]:
## MSA AND ITS FILE CREATION

from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
import Bio.Align.Applications
from Bio import AlignIO

# Putting all of the genbank file into the list
proteins3 = list(SeqIO.parse("3-proteins.gb","gb"))

# So for every protein in the 3-proteins.gb
for protein in proteins3:
    print()
    # Gets the name of the protein
    # So smth like mRNA_1 and so on ...
    name = protein.name
    print(name)
    
    print(len(name)*'-')
          
    # MUSCLE.exe domain that needs to be specified
    muscleExe = r"muscle3.8.31_i86win32.exe"

    try:
        from StringIO import StringIO
    except ImportError:
        from io import StringIO

    # Muscle command line where it asks a FASTA line all of the sequence to be aligned
    # Output is the alignment
    muscle_cline = MuscleCommandline(muscleExe, input = f'{name}_new.fasta', out = f'3-alignment-{name}.fasta')
    # Run command line witht writing the output file
    # Default format of MUSCLE is fasta
    
    # This line is nesecarry, if no such line, the program would crash with an error
    # The result is in stdout
    stdout, stderr = muscle_cline()
    
    # Prints the results
    print(AlignIO.read(f'3-alignment-{name}.fasta', "fasta"))


mRNA_1
------
SingleLetterAlphabet() alignment with 29 rows and 821 columns
--------------------------------------------...--- sp|Q1RKH7|RPOD_RICBR
--------------------------------------------...--- sp|P33451|RPOD_RICPR
--------------------------------------------...--- sp|Q68VQ5|RPOD_RICTY
--------------------------------------------...--- sp|Q92FZ8|RPOD_RICCN
--------------------------------------------...--- sp|Q4UJT1|RPOD_RICFE
--------------------------------------------...--- sp|D5AQI9|RPOD_RHOCB
--------------------------------------------...--- sp|P0CZ15|RPOD_RHOCA
--------------------------------------------...--- sp|P52324|RPOD_CAUVC
--------------------------------------------...--- sp|P33452|RPOD_AGRFC
--------------------------------------------...--- sp|Q59753|RPOD_RHIME
--------------------------------------------...--- sp|Q2K619|RPOD_RHIEC
MPTQKPSKASVKPTKKKVDPVIRKKKAPDGAAAKGAKTGAEEKE...--- sp|P17531|RPOD_MYXXA
--------------------------------------------...SKL sp|P5232

1. COMAPARED WITH THE TARGET PROTEIN SEQUENCE THAT IS ALIGNED IN THE ALIGNMENT AND SAVE RESIDUES IN FILE<p>
2. COMPARED WITH THE ORIGINAL UNALIGNED SEQUENCE AND SAVE RESIDUES IN FILE

In [11]:
# 1. COMAPARED WITH THE TARGET PROTEIN SEQUENCE THAT IS ALIGNED IN THE ALIGNMENT AND SAVE RESIDUES IN FILE

from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import AlignInfo # To generate a trivial 'dumb' consensus
from Bio.Align.AlignInfo import SummaryInfo
## load AlignInfo modules and SummaryInfo submodules
import Bio.SeqFeature as BSF
import re

try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass

# Putting all of the genbank file into the list
proteins3 = list(SeqIO.parse("3-proteins.gb","gb"))

print("Different from consensus sequence:")

for protein in proteins3:
    
    # Name of the protein
    name = protein.id
    print("\n",name)
    
    # Protein sequence
    seq = protein.seq
    #print(seq)
    
    # Alignment files being read base on protein name
    align = AlignIO.read(f'3-alignment-{name}.fasta', "fasta")
    #print(align)

    # Summary of the alignment
    summary = SummaryInfo(align)

    # Create gap consensus sequence
    dumb_cons = summary.gap_consensus(require_multiple=1, threshold=0.6)
    
    # For every sequence in the alignment
    for seq in align:
        
        index = 0
        
        # Is there a sequence that starts with mRNA_
        if re.search("mRNA_", seq.id):
            ogProt = seq.seq
            
            # Comparison of gap consensus sequence with out target protein
            # Results are our positions that do not match
            for ch1, ch2 in zip(ogProt, dumb_cons):

                if ch1 != ch2 and ch1 not in "-X" and ch2 not in "-X":

                    # Creation of GO feature for our protein sequences'
                    # Len vieno protein o ne visu proteins yra tas 1..616 salia BLAST
                    feature = BSF.SeqFeature(BSF.FeatureLocation(index, index + 1), type='mutation')

                    # We add a qualifier 'db_xref' to the GO IDs (Terms) and add it to BLAST feature
                    feature.qualifiers['observed'] = ch1

                    # In the same GO feature we add location/qualifier that is 'evidence'
                    feature.qualifiers['consensus'] = ch2

                    # Then for every protein (not the whole stack of proteins but just every individual protein/mRNA)in the genbank file 
                    # we add those features that have a name GO on the left collum
                    # and we add all of the feature items, I mean all of those ids with the qualifier
                    protein.features.append(feature)
                  
                    print(index, ch1, ch2)
                    
                # Ignoring the gaps; not counting in while counting all of the residues
                if ch1 != "-":
                    index += 1
                            
# Write to genbank file
SeqIO.write(proteins3, "4-proteins.gb","gb")

Different from consensus sequence:

 mRNA_1
18 R K
42 V I
70 D E
163 L I
198 D E
328 Y W
486 G A
529 T N
551 S A
555 A P
604 T S

 mRNA_2

 mRNA_3
2 G A
7 A G
24 A L
25 I V
29 G N
30 R K
35 Y H
50 V L
54 P Y
58 R E
65 E K
69 V I
78 Q L
80 L T
94 S D
98 L V
99 Y L
100 L N
103 N V
104 L I
127 V T
128 R K
133 R S
145 V I
158 D N
160 A I
161 V N
167 F L
175 V I
176 R E
202 V L
207 V L
217 H P
265 A V
273 V I
278 V I
292 R G
295 V I
301 E L
306 V I
310 R S
325 K G
331 P R
337 I V
342 S V

 mRNA_4
317 I V
337 R K
343 F Y
361 M T
374 R K
376 F Y
385 I L
401 E D
431 V I
453 H Y
485 E G
526 S T
527 T P
539 G E
679 K E
699 D G
815 S N

 mRNA_5
9 S P
14 G Y
22 M L
26 I V
32 R K
56 G T
57 I L
75 E I
108 A K
112 E S
116 L A
144 V L
145 M L
174 G A
184 T P
193 I A
203 A P
243 V L
253 L F
262 P A
278 E Q
283 I L
299 S G
312 F V
327 A G
374 V R
378 N Q
382 Q D
385 A D

 mRNA_6
59 R D
71 D T
224 L I
281 Y F
307 R F
328 N P
356 M A
466 E D
490 R E
492 K E
495 S L
498 M H
499 V P
504 S A
506 L V
507 L V


6

In [12]:
# 1. CONTINUATION

from Bio import SeqIO
import Bio.Entrez as BE
import Bio.SearchIO as BSIO
from Bio import SeqFeature
from Bio.SeqFeature import SeqFeature
from Bio import SeqFeature as sf
import Bio.SeqFeature as BSF

# Our updated file
seq = list(SeqIO.parse("4-proteins.gb","gb"))

# For every sequence in the file
for s in seq:
    print()
    
    # Protein/sequence name
    name = s.name
    print(name)
    
    # Finds all of the locations of the residues
    for f in s.features:
        # Check if the feature in genbank file is 'mutation'
        if f.type == "mutation":
            # Get the location that is not similar to consesnus when looking for a similarity between it and the aligned original sequence
            print(f.location)


mRNA_1
[18:19]
[42:43]
[70:71]
[163:164]
[198:199]
[328:329]
[486:487]
[529:530]
[551:552]
[555:556]
[604:605]

mRNA_2

mRNA_3
[2:3]
[7:8]
[24:25]
[25:26]
[29:30]
[30:31]
[35:36]
[50:51]
[54:55]
[58:59]
[65:66]
[69:70]
[78:79]
[80:81]
[94:95]
[98:99]
[99:100]
[100:101]
[103:104]
[104:105]
[127:128]
[128:129]
[133:134]
[145:146]
[158:159]
[160:161]
[161:162]
[167:168]
[175:176]
[176:177]
[202:203]
[207:208]
[217:218]
[265:266]
[273:274]
[278:279]
[292:293]
[295:296]
[301:302]
[306:307]
[310:311]
[325:326]
[331:332]
[337:338]
[342:343]

mRNA_4
[317:318]
[337:338]
[343:344]
[361:362]
[374:375]
[376:377]
[385:386]
[401:402]
[431:432]
[453:454]
[485:486]
[526:527]
[527:528]
[539:540]
[679:680]
[699:700]
[815:816]

mRNA_5
[9:10]
[14:15]
[22:23]
[26:27]
[32:33]
[56:57]
[57:58]
[75:76]
[108:109]
[112:113]
[116:117]
[144:145]
[145:146]
[174:175]
[184:185]
[193:194]
[203:204]
[243:244]
[253:254]
[262:263]
[278:279]
[283:284]
[299:300]
[312:313]
[327:328]
[374:375]
[378:379]
[382:383]
[385:386]


In [13]:
# 2. COMPARED WITH THE ORIGINAL UNALIGNED SEQUENCE AND SAVE RESIDUES IN FILE


from Bio import AlignIO
from Bio import SeqIO
from Bio.Align import AlignInfo # To generate a trivial 'dumb' consensus
from Bio.Align.AlignInfo import SummaryInfo
## load AlignInfo modules and SummaryInfo submodules
import Bio.SeqFeature as BSF

try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass

# Putting all of the genbank file into the list
proteins3 = list(SeqIO.parse("3-proteins.gb","gb"))

print("Different from consensus sequence: \n")

for protein in proteins3:
    
    # Name of the protein
    name = protein.id
    print(name)
    if protein.id == "mRNA_2":
        print("N/A")
    else:
        print("Position calculation starts from 0. In file it starts from 1.")
    
    # Protein sequence
    seq = protein.seq
    #print(seq)
    
    # Alignment files being read base on protein name
    align = AlignIO.read(f'3-alignment-{name}.fasta', "fasta")
    #print(align)

    # Summary of the alignment
    summary = SummaryInfo(align)

    # Create dumb consensus sequence
    dumb_cons = summary.dumb_consensus(threshold=0.6)

    index = 0
    
    # Comparison of dumb consensus sequence with out target protein
    # Results are our positions that do not match
    for n1, n2 in zip(seq,dumb_cons):
        
        if n1 != n2 and n1 not in "-X" and n2 not in "-X":
                
            # Creation of GO feature for our protein sequences'
            # Len vieno protein o ne visu proteins yra tas 1..616 salia BLAST
            feature = BSF.SeqFeature(BSF.FeatureLocation(index,index + 1), type='mutation')

            # We add a qualifier 'db_xref' to the GO IDs (Terms) and add it to BLAST feature
            feature.qualifiers['observed'] = n1

            # In the same GO feature we add location/qualifier that is 'evidence'
            feature.qualifiers['consensus'] = n2

            # Then for every protein (not the whole stack of proteins but just every individual protein/mRNA)in the genbank file 
            # we add those features that have a name GO on the left collum
            # and we add all of the feature items, I mean all of those ids with the qualifier
            protein.features.append(feature)
     
        if n1 != "X":
            index += 1
     
    # The [] snipit is from https://stackoverflow.com/questions/8545492/find-the-position-of-difference-between-two-strings 
    # by ovgolovin
    # modified to ignore ambiguous character X
    results = [i for i, (ch1, ch2) in enumerate(zip(dumb_cons, seq)) if ch1 != ch2 and ch1 != "X" and ch2 not in "X"]
    #print(name, results)

    # The start of our gourped positions
    start = None
    
    # Grouped position; Our positions that are not similar to the consensus
    # Basically [(1,10),(12,20),...] and so on
    grouped = []
    
    # For every index of our positions
    for i, pos in enumerate(results):
        # If we havent started than
        if start is None:
            # Make our start the postion
            start = pos
            
            # Check so it would start at then of our postion list,
            # if not it would an error
            if len(results) == i + 1:
                grouped.append((start, start))
                break

        # Check if next element exists
        if len(results) == i + 1:
            grouped.append((start, pos))
            break
        
        # If the position group has to same numbers like for example (7,7)
        # It means that its that Nth position, or in the examples case, the position is just 7
        next_pos = results[i + 1]
        if pos + 1 != next_pos:
            grouped.append((start, pos))
            start = None
    
    for position in grouped:
        print(position)
    print()
                
# Write to genbank file
SeqIO.write(proteins3, "4-proteins2.gb","gb")

Different from consensus sequence: 

mRNA_1
Position calculation starts from 0. In file it starts from 1.
(1, 10)
(12, 20)
(25, 25)
(36, 37)
(57, 57)
(60, 60)
(78, 79)
(102, 102)
(109, 109)
(111, 123)
(128, 129)
(132, 133)
(135, 136)
(138, 138)
(140, 140)
(143, 145)
(153, 154)
(156, 156)
(158, 158)
(160, 160)
(162, 162)
(164, 166)
(169, 169)
(171, 171)
(175, 175)
(178, 180)
(182, 184)
(188, 188)
(190, 190)
(199, 199)
(203, 209)
(217, 217)
(225, 227)
(229, 234)
(236, 239)
(241, 250)
(254, 257)
(259, 259)
(271, 271)
(281, 282)
(291, 291)
(294, 294)
(296, 299)
(304, 317)
(320, 325)
(366, 366)
(429, 429)
(438, 438)
(447, 447)
(450, 450)
(455, 455)
(470, 470)
(476, 476)
(485, 485)
(488, 488)
(494, 494)
(501, 501)
(512, 513)
(524, 524)
(531, 531)
(535, 535)
(539, 540)
(546, 546)
(548, 548)
(555, 556)
(559, 560)
(562, 569)
(571, 579)
(581, 591)
(593, 606)
(608, 609)
(611, 615)

mRNA_2
N/A

mRNA_3
Position calculation starts from 0. In file it starts from 1.
(2, 2)
(7, 7)
(24, 25)
(29, 30)
(35

6

I did both ways, by comparing the original sequence which was aligned (OA) with the consensus (gapped) and also the original unaligned (OU) sequence with the consensus (dumb).<p>
I have noticed that the OU sequences compared with consensus has a lot of different residues. Most of the residues do not match the consensus. However, while comparing the OA with the consensus it had less difference bewteen them. I think that it is because, for example we are comparing our raw sequnce with the ungapped consensus and it will deffinetaly have a lot of differences exluding the ambiguous characters X. While using OA and gaped consensus it has less differences because they are a bit more well aligned together, because of the gaps in both of the consensus and OA sequences. I think that the gapped consensus keeps the aligments (all the gaps and all).